#Multicore Computing HW4
#####MohammadArman Soleimani 98105835
#####Farzam Zohdi-Nasab 9710????

###Q1: Image inversion

In [1]:
%%writefile img_invert.cu

#include <cuda_runtime.h>
#include <opencv2/core/core.hpp>
#include <opencv2/highgui/highgui.hpp>
#include <opencv2/imgproc/imgproc.hpp>

#define tpb 128

using namespace cv;

__global__ void Inversion(unsigned char* in, unsigned char* out, int total_pixels) {

	int position = blockIdx.x * blockDim.x + threadIdx.x;

	if (position < total_pixels)
	{
		out[position] = 255-in[position];
	}  
}


int main()
{

	Mat img = imread("img.jpg",IMREAD_COLOR);
	Size s = img.size();
	int w = s.width;
	int h = s.height;
	Mat img_invert(h, w, CV_8UC3, Scalar(0,0,0));

	unsigned char* char_img = img.data;
	unsigned char* new_img = img_invert.data;

	int u_char_size = h * w * 3 * sizeof(unsigned char);

	unsigned char *ar_img, *ar_img_inv;

	int vec_size = h * w * 3;
	int block_count = (vec_size + tpb - 1)/tpb;

	cudaMalloc((void**) &ar_img, u_char_size);
	cudaMalloc((void**) &ar_img_inv, u_char_size);

	cudaMemcpy(ar_img, char_img, u_char_size, cudaMemcpyHostToDevice);
	cudaMemcpy(ar_img_inv, new_img, u_char_size, cudaMemcpyHostToDevice);

	Inversion<<<block_count, tpb>>>  (ar_img, ar_img_inv, vec_size);

	cudaMemcpy(char_img, ar_img, u_char_size, cudaMemcpyDeviceToHost);
	cudaMemcpy(new_img, ar_img_inv, u_char_size, cudaMemcpyDeviceToHost);

	cudaFree(ar_img);
	cudaFree(ar_img_inv);
   
	Mat output = Mat(h, w, CV_8UC3, new_img);
	imwrite("inverted.jpg", output);
}

Writing img_invert.cu


In [2]:
%%shell
nvcc img_invert.cu `pkg-config opencv --cflags --libs` -o img_invert

In [4]:
%%shell

./img_invert

###Q2: Green Screen

In [5]:
%%writefile gs.cu

#include <cuda_runtime.h>
#include <opencv2/core/core.hpp>
#include <opencv2/highgui/highgui.hpp>
#include <opencv2/imgproc/imgproc.hpp>

#define tpb 128

using namespace cv;

__global__ void GreenScreen(unsigned char* in, unsigned char* out, unsigned char* bg, int total_pixels) {

	int r = (blockIdx.x * blockDim.x + threadIdx.x)*3;
  int g = r+1;
  int b = r+2;

	if (b < total_pixels)
	{
		int gd = 275 + in[r] + in[b];
    int rd = 255 + in[g] + in[b];
    int bd = 255 + in[r] + in[g];

    if (gd < rd && gd < bd){
      out[r] = bg[r];
      out[g] = bg[g];
      out[b] = bg[b];
    }
    else {
      out[r] = in[r];
      out[g] = in[g];
      out[b] = in[b];
    }
	}  
}


int main()
{

	Mat img = imread("d.jpg",IMREAD_COLOR);
	Size s = img.size();
	int w = s.width;
	int h = s.height;

  Mat bg = imread("bg.jpg",IMREAD_COLOR);
	Mat img_invert(h, w, CV_8UC3, Scalar(0,0,0));

	unsigned char* char_img = img.data;
	unsigned char* new_img = img_invert.data;
  unsigned char* char_bg = bg.data;

	int u_char_size = h * w * 3 * sizeof(unsigned char);

	unsigned char *ar_img, *ar_img_inv, *ar_bg;

	int vec_size = h * w * 3;
	int block_count = ((vec_size + tpb - 1)/tpb)/3 + 1;

	cudaMalloc((void**) &ar_img, u_char_size);
	cudaMalloc((void**) &ar_img_inv, u_char_size);
  cudaMalloc((void**) &ar_bg, u_char_size);

	cudaMemcpy(ar_img, char_img, u_char_size, cudaMemcpyHostToDevice);
	cudaMemcpy(ar_img_inv, new_img, u_char_size, cudaMemcpyHostToDevice);
  cudaMemcpy(ar_bg, char_bg, u_char_size, cudaMemcpyHostToDevice);

	GreenScreen<<<block_count, tpb>>>  (ar_img, ar_img_inv, ar_bg, vec_size);

	cudaMemcpy(char_img, ar_img, u_char_size, cudaMemcpyDeviceToHost);
	cudaMemcpy(new_img, ar_img_inv, u_char_size, cudaMemcpyDeviceToHost);
  cudaMemcpy(char_bg, ar_bg, u_char_size, cudaMemcpyDeviceToHost);

	cudaFree(ar_img);
	cudaFree(ar_img_inv);
  cudaFree(ar_bg);
   
	Mat output = Mat(h, w, CV_8UC3, new_img);
	imwrite("gs.jpg", output);
}

Writing gs.cu


In [6]:
%%shell
nvcc gs.cu `pkg-config opencv --cflags --libs` -o gs

In [7]:
%%shell

./gs

In [8]:
%%writefile sharp.cu

#include <cuda_runtime.h>
#include <opencv2/core/core.hpp>
#include <opencv2/highgui/highgui.hpp>
#include <opencv2/imgproc/imgproc.hpp>

#define tpb 128

using namespace cv;

__global__ void Sharpen(unsigned char* in, unsigned char* out, int total_pixels, int w, int down, int up) {

	int i = blockIdx.x * blockDim.x + threadIdx.x;
  
  if (i > down && i < up){
    if (i%w !=0 && i%w != w-1){
      //find nearby positions
      int up = i - 3*w;
      int down = i + 3*w;
      int left = i - 3;
      int right = i + 3;

      int tmp = 5*in[i]-in[up]-in[down]-in[left]-in[right];
      if(tmp<0) tmp=0;
      if(tmp>255) tmp=255;

      out[i]=tmp;

      //out[i]=in[i];
    }
    else {
      out[i]=in[i];
    }
  }
  else {
    out[i]=in[i];
  }
  
}


int main()
{

	Mat img = imread("gs.jpg",IMREAD_COLOR);
	Size s = img.size();
	int w = s.width;
	int h = s.height;

	Mat img_invert(h, w, CV_8UC3, Scalar(0,0,0));

	unsigned char* char_img = img.data;
	unsigned char* new_img = img_invert.data;

	int u_char_size = h * w * 3 * sizeof(unsigned char);

	unsigned char *ar_img, *ar_img_inv;

	int vec_size = h * w * 3;
	int block_count = ((vec_size + tpb - 1)/tpb) + 1;

	cudaMalloc((void**) &ar_img, u_char_size);
	cudaMalloc((void**) &ar_img_inv, u_char_size);

	cudaMemcpy(ar_img, char_img, u_char_size, cudaMemcpyHostToDevice);
	cudaMemcpy(ar_img_inv, new_img, u_char_size, cudaMemcpyHostToDevice);

  int down = 3*w;
  int up = vec_size - 3*w;

	Sharpen<<<block_count, tpb>>>  (ar_img, ar_img_inv, vec_size, w, down, up);

	cudaMemcpy(char_img, ar_img, u_char_size, cudaMemcpyDeviceToHost);
	cudaMemcpy(new_img, ar_img_inv, u_char_size, cudaMemcpyDeviceToHost);

	cudaFree(ar_img);
	cudaFree(ar_img_inv);
   
	Mat output = Mat(h, w, CV_8UC3, new_img);
	imwrite("sharp.jpg", output);
}

Writing sharp.cu


In [9]:
%%shell

nvcc sharp.cu `pkg-config opencv --cflags --libs` -o sharp

In [10]:
%%shell

./sharp